In [3]:
import numpy as np

import tensorflow as tf

!pip install -q tensorflow_hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version:",tf.__version__)
print("Eager mode:",tf.executing_eagerly())
print("Hub version:",hub.__version__)
print("GPU is","available" if tf.config.experimental.list_physical_devices("GPU") else "not available")

Version: 2.1.0
Eager mode: True
Hub version: 0.8.0
GPU is available


In [5]:
# Split the training set into 60% and 40%, so we'll end up with :
# 15,000 examples for training, 
# 10,000 examples for validation
# 25,000 examples for testing.
train_data,validation_data,test_data=tfds.load(
    name="imdb_reviews",
    split=('train[:60%]','train[60%:]','test'),
    as_supervised=True
)

Shuffling and writing examples to C:\Users\97110\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteVV90T5\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\97110\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteVV90T5\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\97110\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteVV90T5\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\97110\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [11]:
train_example_batch,train_labels_batch=next(iter(train_data.batch(10)))
# print first 10 examples
print(train_example_batch)
print('\n\n')
# print first 10 labels
# 0 for negative
# 1 for positive
print(train_labels_batch)

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

In [32]:
# The saved model can be loaded directly:
embed = hub.load("https://hub.tensorflow.google.cn/google/tf2-preview/gnews-swivel-20dim/1")
embeddings = embed(["cat is on the mat", "dog is in the fog","i love the dog","machine learning"])
embeddings

<tf.Tensor: shape=(4, 20), dtype=float32, numpy=
array([[ 0.8666395 ,  0.35917717,  0.00579667,  0.681002  , -0.54226625,
         0.22343187, -0.38796625,  0.62195706,  0.22117122, -0.48538068,
        -1.2674141 ,  0.886369  , -0.3284907 , -0.13924702, -0.53327686,
         0.5739708 , -0.05905761,  0.13629246, -1.1718255 , -0.31494334],
       [ 0.960218  ,  0.62520486,  0.06261905,  0.37425604,  0.24782333,
        -0.39351934, -0.7418429 ,  0.56599647, -0.26197797, -0.69016844,
        -0.76565284,  0.71412426, -0.4537978 , -0.50701594, -0.8499377 ,
         0.8917156 , -0.30278975,  0.2149126 , -1.1098894 , -0.46719775],
       [ 1.0042577 , -0.54693604,  0.4764572 ,  0.3467536 , -1.4417639 ,
        -0.5856145 , -1.155564  ,  0.83961177,  1.4662061 ,  0.47487593,
        -1.6507809 ,  0.3845137 ,  0.29409006,  0.36355785, -1.4121084 ,
        -0.06935343,  0.59598255,  0.01526355, -0.9195359 , -0.76128733],
       [ 1.8293254 , -0.5837382 ,  0.67903864,  0.38701615, -0.06374515,

In [14]:
# use a pre-trained text embedding model from TensorFlow Hub called google/tf2-preview/gnews-swivel-20dim/1
embedding = "https://hub.tensorflow.google.cn/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [17]:
# build the full model

# The layers are stacked sequentially to build the classifier:
# 1.The first layer is a TensorFlow Hub layer. This layer uses a pre-trained Saved Model to map a sentence into its embedding vector. The pre-trained text embedding model that we are using (google/tf2-preview/gnews-swivel-20dim/1) splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: (num_examples, embedding_dimension).
# 2.This fixed-length output vector is piped through a fully-connected (Dense) layer with 16 hidden units.
# 3.The last layer is densely connected with a single output node.
model=tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(1)
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy']
             )

In [19]:
# Train the model
history=model.fit(train_data.shuffle(10000).batch(512),
                  epochs=20,
                  validation_data=validation_data.batch(512),
                  verbose=1
                 )

Epoch 1/20
30/30 [==============================] - 3s 104ms/step - loss: 0.8235 - accuracy: 0.5089 - val_loss: 0.6862 - val_accuracy: 0.5727
Epoch 2/20
30/30 [==============================] - 2s 64ms/step - loss: 0.6461 - accuracy: 0.6189 - val_loss: 0.6150 - val_accuracy: 0.6497
Epoch 3/20
30/30 [==============================] - 2s 60ms/step - loss: 0.5865 - accuracy: 0.6795 - val_loss: 0.5733 - val_accuracy: 0.6870
Epoch 4/20
30/30 [==============================] - 2s 60ms/step - loss: 0.5386 - accuracy: 0.7133 - val_loss: 0.5343 - val_accuracy: 0.7108
Epoch 5/20
30/30 [==============================] - 2s 63ms/step - loss: 0.4964 - accuracy: 0.7427 - val_loss: 0.4966 - val_accuracy: 0.7492
Epoch 6/20
30/30 [==============================] - 2s 64ms/step - loss: 0.4554 - accuracy: 0.7751 - val_loss: 0.4622 - val_accuracy: 0.7688
Epoch 7/20
30/30 [==============================] - 2s 61ms/step - loss: 0.4177 - accuracy: 0.8053 - val_loss: 0.4337 - val_accuracy: 0.7812
Epoch 8/20
3

In [26]:
# Evaluate the model
results=model.evaluate(test_data.batch(512),verbose=1)

print('\n')
for name,value in zip(model.metrics_names,results):
    print("%s: %.3f"%(name,value))

     49/Unknown - 2s 33ms/step - loss: 0.3240 - accuracy: 0.8566

loss: 0.324
accuracy: 0.857
